# Colonoscopy Polyp Detection w/Faster R-CNN and Lightning

## imports and setup

In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import torch

In [2]:
CONFIG = dict (
    project = "hlc-polyp-detection",
    architecture = "fasterrcnn_resnext50_32x4d",
    dataset_id = "hlc-custom-polyp-detection",
    infra = "osx",
    num_classes = 2,
    lr=0.01,
    min_lr=0.0000001,
    epochs=15,
    batch_size=4,
    nesterov=True,
    momentum=0.9,
    weight_decay=0.0005,
    clip_limit=0.25,
    difference=False,
)

ROOT_DIR = os.path.abspath("./")
DATA_DIR = os.path.join(ROOT_DIR, "data")
MODEL_DIR = os.path.join(ROOT_DIR, "model")
LOG_DIR = os.path.join(ROOT_DIR, "log")

NUM_CLASSES = 2

BATCH_SIZE = 4
INPUT_SIZE = 800
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 4

wandb_key = Path(os.path.join(ROOT_DIR, "wandb.txt")).read_text().strip()
os.environ["WANDB_API_KEY"] = wandb_key

In [3]:
if CONFIG["infra"] == "paperspace":
    !pip install -r alubumentations pytorch-lightning wandb --upgrade
    import wandb
    !nvidia-smi
else:
    import wandb

In [4]:
from contextlib import contextmanager

@contextmanager
def wandb_context(configuration=CONFIG):
    run = wandb.init(reinit=True, config=configuration, project=CONFIG['project'])
    try:
        yield run
    finally:
        wandb.finish()

In [32]:
from src.PolypsPLDataModule import PolypsPLDataModule

polyp_dm = PolypsPLDataModule(data_dir=DATA_DIR, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

def train_model(model, run_name, dm=None, run=None):
    wandb_logger = None
    if run is not None:
        run.config["train_run_name"] = run_name

        from pytorch_lightning.loggers import WandbLogger
        wandb_logger = WandbLogger()
    
    chkpt = ModelCheckpoint(
        dirpath=os.path.join(MODEL_DIR, "checkpoints"),
        filename=f"chkpt-{run_name}",
        monitor="val_recall",
        mode="max")
    
    trainer = Trainer(gpus=1, logger=wandb_logger, callbacks=[chkpt]) if DEVICE == "cuda" else Trainer(accelerator="cpu", logger=wandb_logger, callbacks=[chkpt])
    trainer.fit(
        model,
        datamodule=dm)
    
    return LightningFasterModule.load_from_checkpoint(chkpt.best_model_path)

In [34]:
from src.LightningFasterModule import LightningFasterModule

with wandb_context() as run:
    model = LightningFasterModule(CONFIG)
    if run is not None:
        run.watch(model)
    trained_model = train_model(model, run_name="head", dm=polyp_dm, run=run)

    model.full_train()
    model = train_model(model, run_name="full", dm=polyp_dm, run=run)

/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(

  | Name 

Epoch 00000: adjusting learning rate of group 0 to 1.0000e-02.


Sanity Checking: 0it [00:00, ?it/s]

IN VALIDATION STEP
type(targets)=<class 'tuple'> type(pred_boxes)=<class 'list'>
pred_boxes=[{'boxes': tensor([[0.0000e+00, 0.0000e+00, 6.3554e+02, 7.2572e+02],
        [6.2482e+02, 4.3144e+02, 6.4298e+02, 4.7503e+02],
        [1.6064e+02, 2.9829e+02, 3.8609e+02, 5.2213e+02],
        [1.5455e+02, 3.6195e+02, 6.5928e+02, 7.9700e+02],
        [3.7735e+02, 5.5647e+02, 5.1888e+02, 5.9885e+02],
        [9.7314e+01, 3.0929e+02, 3.5015e+02, 5.9012e+02],
        [3.1212e+02, 6.4413e+02, 3.8753e+02, 7.2184e+02],
        [1.5536e+02, 2.8107e+02, 3.1475e+02, 3.7091e+02],
        [5.6991e+02, 4.0497e+02, 6.0948e+02, 4.4935e+02],
        [2.8840e+02, 6.1680e+02, 4.0556e+02, 7.3888e+02],
        [2.3704e+02, 3.7959e+02, 2.8094e+02, 4.7032e+02],
        [2.8269e+02, 5.3860e+02, 4.0512e+02, 7.1697e+02],
        [2.7192e+02, 5.3787e+02, 5.4549e+02, 6.1348e+02],
        [1.5601e+02, 2.8373e+02, 2.5231e+02, 4.6804e+02],
        [6.2832e+02, 2.5529e+02, 6.9471e+02, 3.1191e+02],
        [1.9076e+02, 2.7859

Training: 0it [00:00, ?it/s]

IndexError: index is out of bounds for dimension with size 0